### All needed imports

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


### Limit GPU Usage

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-08 09:23:26.753686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-08 09:23:26.769775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-08 09:23:26.769919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-08 09:23:26.831812: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Preprocessing Data for Model 1

#### Video Paths

In [52]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

test_falling_paths = [
                "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_default_paths = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"
]

#### Processing code, returns frame (num_frames, 224,224,3) and frame_diffs (num_frames, 224,224)

In [4]:
# Code to apply the same random transformation 

In [49]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
        # print(len(labels))
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [50]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    # frame_diffs dimension
    # print(frame_diffs.shape)
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    # frame_diffs_expanded dimension
    # print(frame_diffs_expanded.shape)
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    # combined_input dimension
    # print(combined_input.shape)
    
    # Visualize the difference
    # cv2.imwrite('img_still.png', combined_input[10])
    return combined_input

In [54]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
test_still_frames, test_still_diff, test_still_labels = process_videos(test_default_paths, 0)
print(still_frames.shape)
falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)
test_falling_frames, test_falling_diff, test_falling_labels = process_videos(test_falling_paths, 1)
print(falling_frames.shape)

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)

# Combine them
test_concatenate_frames = np.concatenate([test_still_frames, test_falling_frames], axis = 0)
test_concatenate_diff =  np.concatenate([test_still_diff, test_falling_diff], axis = 0)
test_concatenate_labels =  np.concatenate([test_still_labels, test_falling_labels], axis = 0)
# print(test_concatenate_frames.shape)
# Create 224x224x4 shape for the model
test_combined_input = combine_frames_and_diffs(test_concatenate_frames, test_concatenate_diff)
print(combined_input.shape)

# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
concatenate_labels = concatenate_labels[indices]

#data augmentation 
data_augmentation = tf.keras.Sequential([
    # layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(factor=0.2)

])


(567, 224, 224, 3)
(513, 224, 224, 3)
(1080, 224, 224, 4)


In [8]:
# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_combined_input, test_concatenate_labels))
    test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

#### Create the model

In [9]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')

    # Apply data augmentation here
    x = data_augmentation(combined_input)

    # Convolutional layers for image processing
    # Convolutional layers are useful for learning spatial hierarchies and detecting features 
    x = Conv2D(32, (3, 3), activation='relu')(x) #  Convolutional layers extract spatial features from the image
    x = MaxPooling2D((2, 2))(x) #  Max-pooling layers reduce the size of the feature maps.
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Fully connected layers: These layers learn to make decisions based on the features extracted by the convolutional layers
    x = Flatten()(x) # This line flattens the 2D feature maps into a 1D vector
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss
    
    return model
    
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 combined_input (InputLayer)  [(None, 224, 224, 4)]    0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 4)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                         

In [10]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5


2023-08-08 09:23:35.896112: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8600
2023-08-08 09:23:36.303155: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


54/54 [==============================] - 5s 43ms/step - loss: 18.1312 - accuracy: 0.7130 - val_loss: 0.3299 - val_accuracy: 0.9306
Epoch 2/5
54/54 [==============================] - 2s 40ms/step - loss: 0.3988 - accuracy: 0.8993 - val_loss: 0.6048 - val_accuracy: 0.8241
Epoch 3/5
54/54 [==============================] - 2s 40ms/step - loss: 0.3744 - accuracy: 0.8924 - val_loss: 0.4778 - val_accuracy: 0.8380
Epoch 4/5
54/54 [==============================] - 2s 40ms/step - loss: 0.3048 - accuracy: 0.9225 - val_loss: 0.3582 - val_accuracy: 0.8611
Epoch 5/5
54/54 [==============================] - 2s 40ms/step - loss: 0.2870 - accuracy: 0.8762 - val_loss: 0.1898 - val_accuracy: 0.9444


#### Save the model

In [11]:
model.evaluate(test_dataset)
model.save("m1")

64/64 [==============================] - 2s 12ms/step - loss: 0.6166 - accuracy: 0.7034


2023-08-08 09:23:51.283876: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: m1/assets


#### Quantize and save as tflite

In [12]:
def representative_dataset():
    for data, label in tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels)).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_m1 = converter.convert()

# Save the quantized model 1
with open('m1_quant.tflite', 'wb') as f: f.write(tflite_quant_m1)

INFO:tensorflow:Assets written to: /tmp/tmpftllhyf0/assets


/home/sylvia/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-08 09:23:54.524303: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-08 09:23:54.524325: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-08 09:23:54.524851: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpftllhyf0
2023-08-08 09:23:54.527436: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-08 09:23:54.527448: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpftllhyf0
2023-08-08 09:23:54.538620: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-08-08 09:23:54.64866

#### Pass in video for model 1 to predict to generate data for model 2

In [23]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="m1_quant.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
print('Input shape:', input_details[0]['shape'])
output_details = interpreter.get_output_details()
print('Output shape:', output_details[0]['shape'])

# [1,1] u get [[1]], instead of [1]
# idk why it wraps an extra bracket but it just does

Input shape: [  1 224 224   4]
Output shape: [1 1]


#### Predict with model 1 and save the results into an array

In [20]:
def process_single_vid(path):
    frames = []
    frame_diffs = []
    video_cap = cv2.VideoCapture(path)
    
    prev_gray_frame = None
    
    while video_cap.isOpened():
        ret, frame = video_cap.read()
        
        if not ret:
            break
        
        # Resize and convert frame to RGB
        frame_resized = cv2.resize(frame, (224, 224))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        
        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

        if prev_gray_frame is None:
            frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
        else:
            frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
            # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
        
        prev_gray_frame = gray_frame
        
        frames.append(frame_rgb)
        frame_diffs.append(frame_diff)

    video_cap.release()
    return np.array(frames), np.array(frame_diffs)

def predict_single_vid(interpreter, input_data):
    # Get the input and output details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Initialize an empty list to store the results
    results = []

    # Iterate over each instance in the input data
    for instance in input_data:
        # Set the tensor to point to the input data to be inferred
        interpreter.set_tensor(input_details['index'], np.expand_dims(instance, axis=0))

        # Run the computation
        interpreter.invoke()

        # Get the output tensor
        output = interpreter.get_tensor(output_details['index'])

        # Append the output to the results list
        results.append(output)

    # Convert the results list to an array and return it
    return np.array(results)

In [58]:
result_8 = []
labels = []
for vid in falling_paths:
    vid_frames, vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    combined = combine_frames_and_diffs(vid_frames, vid_diffs)
    # convert datatype from float 32 to uint8
    combined = combined.astype(np.uint8)
    # print(combined)
    # predict_single_vid
    res = predict_single_vid(interpreter, combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        result_8.append(window)
        labels.append(1)

for vid in default_paths:
    vid_frames, vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    combined = combine_frames_and_diffs(vid_frames, vid_diffs)
    # convert datatype from float 32 to uint8
    combined = combined.astype(np.uint8)
    # predict_single_vid
    res = predict_single_vid(interpreter, combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        result_8.append(window)
        labels.append(0)

labels = np.array(labels)
result_8 = np.array(result_8)

(870, 8, 1, 1)
(870,)
(870, 8)


In [72]:
print(result_8.shape)
print(labels.shape)
result_8 = result_8.reshape(len(result_8), 8)
print(result_8.shape)

(870, 8)
(870,)
(870, 8)


In [69]:
X = result_8
Y = labels

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset = dataset.shuffle(100).batch(16)

train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

#### Prepare testing data

In [71]:
test_result_8 = []
test_labels = []

for vid in test_falling_paths:
    test_vid_frames, test_vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    test_combined = combine_frames_and_diffs(test_vid_frames, test_vid_diffs)
    # convert datatype from float 32 to uint8
    test_combined = test_combined.astype(np.uint8)
    # print(combined)
    # predict_single_vid
    test_res = predict_single_vid(interpreter, test_combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        test_result_8.append(window)
        test_labels.append(1)

for vid in test_default_paths:
    test_vid_frames, test_vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    test_combined = combine_frames_and_diffs(test_vid_frames, test_vid_diffs)
    # convert datatype from float 32 to uint8
    test_combined = test_combined.astype(np.uint8)
    # predict_single_vid
    test_res = predict_single_vid(interpreter, test_combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        test_result_8.append(window)
        test_labels.append(0)

test_result_8 = np.array(test_result_8)
test_labels = np.array(test_labels)


(660, 8, 1, 1)
(660,)


ValueError: cannot reshape array of size 6960 into shape (660,8)

In [73]:
print(test_result_8.shape)
print(test_labels.shape)
test_result_8 = test_result_8.reshape(len(test_result_8), 8)
print(test_result_8.shape)

(660, 8, 1, 1)
(660,)
(660, 8)


In [75]:
test_X = test_result_8
test_Y = test_labels
# Create the dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_Y))
test_dataset = test_dataset.shuffle(100).batch(16)

(660, 8)


#### Create model 2

In [77]:
def create_model_2():
    # Assuming the 8 uint8 values are flattened into a 1D array
    input_shape = (8, )

    # Define the model
    input_layer = Input(shape=input_shape, dtype='uint8', name='input_layer')

    # Dense layers to process the sequence
    x = Dense(16, activation='relu')(input_layer)
    x = Dense(8, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model_2 = create_model_2()
model_2.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 8)]               0         
                                                                 
 dense_12 (Dense)            (None, 16)                144       
                                                                 
 dense_13 (Dense)            (None, 8)                 136       
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


#### Prepare Tensorflow dataset

In [78]:
model_2.fit(train_dataset, validation_data=val_dataset, epochs=7)

Epoch 1/7
44/44 [==============================] - 0s 3ms/step - loss: 2.6422 - accuracy: 0.7060 - val_loss: 4.3578 - val_accuracy: 0.5904
Epoch 2/7
44/44 [==============================] - 0s 2ms/step - loss: 2.4475 - accuracy: 0.7486 - val_loss: 3.5861 - val_accuracy: 0.6506
Epoch 3/7
44/44 [==============================] - 0s 2ms/step - loss: 1.7686 - accuracy: 0.8068 - val_loss: 2.5952 - val_accuracy: 0.8072
Epoch 4/7
44/44 [==============================] - 0s 6ms/step - loss: 1.6000 - accuracy: 0.8125 - val_loss: 2.5277 - val_accuracy: 0.7711
Epoch 5/7
44/44 [==============================] - 0s 6ms/step - loss: 1.3936 - accuracy: 0.8295 - val_loss: 2.5025 - val_accuracy: 0.8193
Epoch 6/7
44/44 [==============================] - 0s 6ms/step - loss: 1.0758 - accuracy: 0.8537 - val_loss: 2.0907 - val_accuracy: 0.8072
Epoch 7/7
44/44 [==============================] - 0s 8ms/step - loss: 1.0929 - accuracy: 0.8551 - val_loss: 1.5270 - val_accuracy: 0.8494


In [82]:
model_2.evaluate(test_dataset)

 1/42 [..............................] - ETA: 0s - loss: 5.1426 - accuracy: 0.3125

42/42 [==============================] - 0s 1ms/step - loss: 3.9765 - accuracy: 0.5000


[3.9764597415924072, 0.5]